# K-Means Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

We might want to take a look at the distribution of the Title+Body length feature we used before and instead of using the raw number of words create categories based on this length: short, longer,..., super long.

In the questions below I'll refer to length of the combined Title and Body fields as Description length (and by length we mean the number of words when the text is tokenized with pattern="\W").

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import RegexTokenizer, VectorAssembler
from pyspark.ml.clustering import KMeans
# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read the dataset and extract description length features
# 3) write code to answer the quiz questions

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Description Length Features

In [5]:
df = df.withColumn("Desc", concat(col("Title"), lit(' '), col("Body")))

In [6]:
regexTokenizer = RegexTokenizer(inputCol="Desc", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [7]:
body_length = udf(lambda x : len(x), IntegerType())
df = df.withColumn("DescLength", body_length(df.words))

In [8]:
assembler = VectorAssembler(inputCols=["DescLength"], outputCol="DescVec")
df = assembler.transform(df)

In [9]:
number_of_tags = udf(lambda x : len(x.split(" ")), IntegerType())
df = df.withColumn("NumTags", number_of_tags(df.Tags))

# Question 1
How many times is the Description length of the longest question longer than the shortest one's?

Tip: Don't forget to import Spark SQL's aggregate functions that can operate on DataFrame columns.

In [12]:
# TODO: write your code to answer this question
df.agg(min("DescLength")).show()

+---------------+
|min(DescLength)|
+---------------+
|             10|
+---------------+



In [13]:
df.agg(max("DescLength")).show()

+---------------+
|max(DescLength)|
+---------------+
|           7532|
+---------------+



In [14]:
# alternatively
from pyspark.sql import functions as F
df.agg(F.min("DescLength"), F.max("DescLength")).show()

+---------------+---------------+
|min(DescLength)|max(DescLength)|
+---------------+---------------+
|             10|           7532|
+---------------+---------------+



# Question 2
What is the mean and standard deviation of the Description length?

In [15]:
# TODO: write your code to answer this question
df.agg(avg("DescLength"), stddev("DescLength")).show()

+---------------+-----------------------+
|avg(DescLength)|stddev_samp(DescLength)|
+---------------+-----------------------+
|      180.28187|       192.108195335051|
+---------------+-----------------------+



# Question 3
Let's use K-means to create 5 clusters of Description lengths. Set the random seed to 42 and fit a 5-class K-means model on the Description length column (you can use KMeans().setParams(...) ). What length is the center of the cluster representing the longest questions?

In [16]:
# TODO: write your code to answer this question
kmeans = KMeans().setParams(featuresCol="DescVec", predictionCol="DescGroup", k=5, seed=42)
model = kmeans.fit(df)
df = model.transform(df)

In [17]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', Desc="How to check if an uploaded file is an image without mime type? <p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an imag

In [18]:
df.groupby("DescGroup").agg(avg(col("DescLength")), avg(col("NumTags")), count(col("DescLength"))).orderBy("avg(DescLength)").show()

+---------+------------------+------------------+-----------------+
|DescGroup|   avg(DescLength)|      avg(NumTags)|count(DescLength)|
+---------+------------------+------------------+-----------------+
|        1|104.36970233168755| 2.768843288906647|            70078|
|        0| 276.1014773552918| 3.127431985145717|            24774|
|        3| 610.3441249717002|3.2485850124518905|             4417|
|        4|1349.5467741935483|3.2903225806451615|              620|
|        2|3075.8558558558557|3.5945945945945947|              111|
+---------+------------------+------------------+-----------------+

